In [ ]:
import pandas as pd
import FWI
import matplotlib.pyplot as plt
import numpy as np

def total_precipitation(initial_date, final_date, dmet):
    dmet['hourly.time'] = pd.to_datetime(dmet['hourly.time'])
    dmet_copy = dmet.copy()
    dmet_copy.set_index('hourly.time', inplace=True)
    selected_rows = dmet_copy.loc[initial_date:final_date]
    total_precipitation = selected_rows['hourly.precipitation'].sum()

    return total_precipitation

In [ ]:
dmet = pd.read_csv('Samples/2015/2015.csv')
start_index = 0 + 1
LAT = 40.20875
FFMCPrev = 94.29084014892578
DMCPrev = 65.2578125
DCPrev = 370.92425537109375

mts = {5: 31,
       6: 30,
       7: 31,
       8:31, 
       9:30, 
       10:31}

fwix_values = [None] * len(dmet)
ffmc_values = [None] * len(dmet)
dmc_values = [None] * len(dmet)
dc_values = [None] * len(dmet)
isi_values = [None] * len(dmet)
bui_values = [None] * len(dmet)

for i in range(start_index, len(dmet)):
    MONTH = (dmet.iloc[i]['hourly.time']).split('-')[1]
    DAY = (dmet.iloc[i]['hourly.time']).split('-')[2].split('T')[0]

    print(MONTH, DAY)

    TEMP = dmet.iloc[i]['hourly.temperature_2m']
    RH = dmet.iloc[i]['hourly.relative_humidity_2m']
    WIND = dmet.iloc[i]['hourly.wind_speed_10m']


    if(int(DAY) == 1):
        LAST_DAY = mts[int(MONTH) - 1]
        LAST_MONTH = str(int(MONTH) - 1)
        
        if(int(LAST_MONTH) < 10):
            LAST_MONTH = '0' + str(int(LAST_MONTH))
            
        initial_date = f'2015-{LAST_MONTH}-{LAST_DAY}T12:00'
        final_date = f'2015-{MONTH}-{DAY}T12:00'
    elif((int(DAY) < 10) and (int(DAY) > 1)):
        id = '0' + str(int(DAY)-1)
        fd = '0' + str(int(DAY))

        initial_date = f'2015-{MONTH}-{id}T12:00'
        final_date = f'2015-{MONTH}-{fd}T12:00'
    else:
        id = str(int(DAY)-1)
        fd = str(int(DAY))

        initial_date = f'2015-{MONTH}-{id}T12:00'
        final_date = f'2015-{MONTH}-{fd}T12:00'

    
    RAIN = total_precipitation(initial_date, final_date)
    
    FFMCPrev = FWI.FFMC(TEMP,RH,WIND,RAIN,FFMCPrev)
    DMCPrev = FWI.DMC(TEMP,RH,RAIN,DMCPrev,LAT,int(MONTH))
    DCPrev = FWI.DC(TEMP,RAIN,DCPrev,LAT,int(MONTH))
    isi = FWI.ISI(WIND, FFMCPrev)
    bui = FWI.BUI(DMCPrev,DCPrev)
    fwix = FWI.FWI(isi, bui)

    fwix_values[i] = fwix
    ffmc_values[i] = FFMCPrev
    dmc_values[i] = DMCPrev
    dc_values[i] = DCPrev
    isi_values[i] = isi
    bui_values[i] = bui


# Add the fwix_values list as a new column to the DataFrame
dmet_filtered['fwix'] = fwix_values
dmet_filtered['ffmc'] = ffmc_values
dmet_filtered['dmc'] = dmc_values
dmet_filtered['dc'] = dc_values
dmet_filtered['isi'] = isi_values
dmet_filtered['bui'] = bui_values


dmet_filtered.to_csv('Samples/CopernicusToOpenMeteo/nearestStationbyMidDayFWI.csv', index=False)

